# On Ubuntu:

    $ apt-get install libgmp-dev build-essential python-dev python-pip
    $ pip install seccure
    $ pip install cuckoopy
    
## Database mysql queries

CREATE DATABASE seccure;

USE seccure;

CREATE TABLE log (user_id VARCHAR(20), timestamp VARCHAR(20), source_ip VARCHAR(20), dest_ip VARCHAR(20), source_mac VARCHAR(20), dest_mac VARCHAR(20), source_port VARCHAR(20), dest_port VARCHAR(20), enc_log VARBINARY(600), hash_log VARBINARY(400));

SHOW COLUMNS FROM log;

SELECT * FROM log;

In [1]:
import seccure
import hmac
import hashlib
import uuid
import re
import pymysql
import pymysql.cursors
import time
from cuckoopy import CuckooFilter

In [124]:
def textfetch(filename):
    with open(filename) as fp:
        lines = fp.read().split("\n")
    return lines

def concateEncrytionFields(enc_conc_list):
    enc_log = [x for i,x in enumerate(enc_conc_list) if i == enc_conc_list.index(x)]
    concate_log = ''
    for i in enc_log:
        concate_log += i
    return concate_log

def generate_keypair():    
    privkey, pubkey = seccure.generate_keypair(curve='secp160r1')
    return privkey, pubkey

def encrypt(message, publickey):
    ciphertext = seccure.encrypt(message, publickey)
    return ciphertext

def decrypt(ciphertext, privatekey):
    decryptedtext = seccure.decrypt(ciphertext, privatekey)
    return decryptedtext

def compareEncryptDecrypt(message, ciphertext):
    if(message == ciphertext): return 1
    return 0

def hashing(text):
    hash_object = hashlib.sha512(text)
    hex_dig = hash_object.hexdigest()
    return hex_dig

'''
def publickeyGenerator(privatekey):
    publickey = str(seccure.passphrase_to_pubkey(privatekey))
    return publickey

def passphrase_to_hash(passphrase):
    #Converts a passphrase to a hash.
    return hashlib.sha256(passphrase).digest()
    
def hashingMD5(password):
    #using MD5
    hash_object = hashlib.md5(b'Password')
    hex_dig = hash_object.hexdigest()
    print("hashed password using MD5:\n", hex_dig)
    return hash_object

def hash_password(password):
    # uuid is used to generate a random number
    salt = uuid.uuid4().hex
    return hashlib.sha256(salt.encode() + password.encode()).hexdigest() + ':' + salt

def check_password(hashed_password, user_password):
    password, salt = hashed_password.split(':')
    return password == hashlib.sha256(salt.encode() + user_password.encode()).hexdigest()
'''

#Regex functions

def find_timestamp(f1):
    ts_pattern = re.compile('^[0-9]{1,3}[.][0-9]{9}$')
    ts = re.findall(ts_pattern, f1)
    return ts

def find_ip(f1) :
    if f1 == 'Broadcast':
        return f1
    ip_pattern = re.compile('(?:^|\b(?<!\.))(?:1?\d?\d|2[0-4]\d|25[0-5])(?:\.(?:1?\d?\d|2[0-4]\d|25[0-5])){3}(?=$|[^\w.])')
    ip = re.findall(ip_pattern, f1)
    return ip

def find_mac(f1) :
    mac_pattern = re.compile('^(([a-fA-F0-9]{2}[:]){5}([a-fA-F0-9]){2})|([a-zA-Z]{8}[_]([a-fA-F0-9]{2}[:]){2}[a-fA-F0-9]{2})$')
    mac = re.search(mac_pattern, f1)
    return mac

#Insert log details in database:

def updatelog(log_list):
    # Connect to the database. autocommit=True
    connection = pymysql.connect("localhost","root","seccure@7","seccure")
    
    try:
        with connection.cursor() as cursor:
            # Create a new record
            sql = "INSERT INTO `log` (`user_id`, `timestamp`, `source_ip`, `dest_ip`, `source_mac`, \
                                    `dest_mac`, `source_port`, `dest_port`, `enc_log`, `hash_log`) \
                                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

            cursor.execute(sql, (log_list))
            print("Log successfully inserted in database.")

        connection.commit()
    finally:
        connection.close()
        
#Cuckoo filter:
def CuckooSearch(cf,key):
    # Initialize a cuckoo filter with 10000 buckets with bucket size 4 and fingerprint size of 1 byte
    #cf = CuckooFilter(capacity=10000, bucket_size=4, fingerprint_size=1)
    a = time.time()
    cf.insert(key)
    cf.contains(key)
    b = time.time()
    search_time = (b-a)*1000
    print("Searching time in ms: ", search_time)

def CuckooInsert(cf,key):
    # Initialize a cuckoo filter with 10000 buckets with bucket size 4 and fingerprint size of 1 byte
    a = time.time()
    cf.insert(key)
    b = time.time()
    insert_time = (b-a)*1000
    print("Insertion time in ms: ", insert_time)

In [133]:
def main():
    
    filepath = 'log.txt'  
    user_id = 1
    timestamp = ''
    log_list = []
    enc_conc_list = []
    s = []
    lines = textfetch(filepath)
    n = len(lines)
    port_flag = 0
    flag1 = 0
    flag2 = 0
    flag_done = 0
    curr_ip_count = 1
    curr_ts_count = 1
    curr_mac_count = 1
    date_flag = 0
    
    #Initialiing a cuckoo filter
    cf = CuckooFilter(capacity=10000, bucket_size=4, fingerprint_size=1)
    
    #iterate through lines
    for i in range(n):
        
        words = lines[i].split()
        
        for w in words:
            ts = find_timestamp(w)
            ip = find_ip(w)
            mac = find_mac(w)
            if w == 'Date:':
                date_flag =1
                continue
            if date_flag:
                new_date = w
                
            if ts and curr_ts_count <=1:
                log_list.append(str(user_id))
                curr_ts_count +=1
                user_id +=1
                timestamp = ts[0]
                log_list.append(timestamp)
            
            elif ip and curr_ip_count <=2:
                log_list.append(ip[0])
                curr_ip_count += 1
                enc_conc_list.append(ip[0])
                
            elif mac and curr_mac_count <=2:
                log_list.append(mac.group(0))
                enc_conc_list.append(mac.group(0))
                curr_mac_count +=1
            
            if w == 'Src':
                flag1 = 1
                continue
                
            if w == 'Port:'and flag_done == 0:
                port_flag = 1
                flag_done = 1             
                continue
            elif w == 'Port:'and flag_done == 1:
                continue
                
            if port_flag and flag1:
                src_port = w
                log_list.append(src_port)
                enc_conc_list.append(src_port)
                flag1 = 0
                port_flag = 1
                continue
                
            if w == 'Dst':
                flag2 = 1
                continue
                
            if port_flag and flag2:
                dest_port = w
                log_list.append(dest_port)
                enc_conc_list.append(dest_port)
                enc_conc_log = concateEncrytionFields(enc_conc_list)
                
                #encryption
                prikey, pubkey = generate_keypair()               
                byte_enc_log = bytes(enc_conc_log.encode('utf-16'))
                a = time.time()
                enc_log = encrypt(byte_enc_log, pubkey)
                b = time.time()
                encrpytion_time = (b-a)*1000
                print("Encryption time in ms: ", encrpytion_time)
                log_list.append(str(enc_log))
                #byte_dec_log = decrypt(enc_log, prikey)
                #dec_log = str(byte_dec_log.decode('utf-16'))
                # dec_log == enc_log , confirmed
                
                #hashing
                log_chain = 1
                hash_input = str(enc_log) + str(log_chain)
                log_chain += 1
                hash_input = hash_input.encode('utf-16')
                a = time.time()
                hash_log = hashing(hash_input)
                b = time.time()
                hashing_time = (b-a)*1000
                print("Hashing time in ms: ", hashing_time)
                log_list.append(hash_log)
                s = [x for i, x in enumerate(log_list) if i == log_list.index(x)]           
                #update previous log to the database
                print(s)
                updatelog(s)
                curr_ip_count = 1
                curr_ts_count = 1
                curr_mac_count = 1
                log_list = []
                enc_conc_list = []
                s = []
                flag2 = 0
                port_flag = 0
                flag_done = 0
                break

#main function
if __name__ == '__main__':
    main()

Encryption time in ms:  7.325887680053711
Hashing time in ms:  0.025987625122070312
['1', '0.000000000', '10.0.47.2', '10.0.47.1', 'fa:16:3e:94:7c:0d', 'f4:4d:30:49:1e:01', '68', '67', 'b\'\\x016\\t\\xc2\\xab\\x9e\\xe1\\x9d\\x9bqL6\\xb4D\\x8b\\x9f\\x15\\x9a\\x1eX\\xd0/\\xeafaN\\xec\\xf9\\xd3\\x1dgm!\\xbd\\xeb\\x96\\x89\\xfb1[5\\x98\\xd8\\xba\\x99\\xb1\\x08\\xf5\\x04\\n\\xf2\\x1d\\xe1\\x9c]7 ;o)\\x0c\\xdd\\x9f\\x98,\\xf1\\x08\\x8aX\\xab%0\\xeb\\xffA\\xe0a\\x14\\x98\\xf4c\\x02yED:\\xa5~\\x88\\xcd\\x9b\\x06t\\x98\\xca\\x90\\xc5\\xd5\\xc4U\\\'K\\xadq\\xca*K\\x16\\x1b^\\xd4\\xcd\\xd5/J\\xad\\xb6z\\xfa?\\x89\\xd4R\\xa6/\\xe8\\x98\\xd2\\xaa\\x82B\\xe0\\x8dK]\\xe2z\\x04Y\\x00\\xef\\x1e\\xcb"\\xfb\'', '91c9d75dee2fa352dc884fe49701ef988331f43edab702c24c75e83947ca01fb5c6ceea196b62e4b5e0429e4631c34dedc8f0065a6e506c5d65c9237bbc06295']
Log successfully inserted in database.
Encryption time in ms:  4.321098327636719
Hashing time in ms:  0.015020370483398438
['2', '0.000289000', '10.0.47.1', '10.0.47.

Hashing time in ms:  0.02002716064453125
['11', '16.823675000', '192.168.81.151', '172.217.17.142', 'f4:4d:30:49:1e:01', 'BestItWo_01:d1:64', '51904', '443', "b'\\x010\\xd60\\x92r\\xd7\\x11F5F\\xad\\xe4\\x9a\\xe6`\\xe6%\\x10YH}\\xc0\\x9ch\\xf2e\\xec\\x83\\x83\\xbeo\\xb4\\xf5\\xed\\xdc|9\\xd7O\\x00F\\xa0\\x0c\\xfb\\t\\xe9#\\xa4\\xd5<#n\\xb7\\xd7\\xdd\\x99\\xde.\\x894Oi\\xf9\\x02\\xb5_Q\\x8aH\\xf4v\\xc6)\\xcb\\x88\\xbf\\xad\\x06]5\\xa0\\x80\\xc4\\xdc\\x80V\\xc9\\xfc\\x8c\\xfa\\xa7\\x04\\xe1\\x85\\x98\\xca\\xa7\\xe9\\x17\\x17U6\\x04\\xe3\\xfe\\xa7s\\x06\\xf9\\xf5f39\\xf4\\xa6\\x9d@\\xf2\\xcb\\xfcb\\xfc\\xe6m\\xe9(9)]\\x00\\x16)\\xc9N\\x9b\\xff1\\xc1\\xf2\\xce\\x86L6\\x02\\xe5\\xd3\\xe2\\xf7\\xb27\\x03\\x13A+Q\\x07\\x00\\xe5\\xe9\\xefq\\x06\\xcbX,\\xb0\\xfan\\x80\\xf9@Z'", 'ae9b1b4ce23b0fc74ed488e31e0c024f84eb29a9afcded9b5993eadc8b8388b209ba3bf6d0d0aaa4739311f440e00d5097f90e5a3c8249864535641f987d4d57']
Log successfully inserted in database.
Encryption time in ms:  14.027118682861328
Hashin

Log successfully inserted in database.
Encryption time in ms:  5.624055862426758
Hashing time in ms:  0.010967254638671875
['28', '41.644261000', '66.218.84.140', '192.168.81.151', 'BestItWo_01:d1:64', 'f4:4d:30:49:1e:01', '443', '41328', 'b\'\\x009\\xd5\\x18\\x0fA\\xcb\\x15\\xbdK\\xc3\\xbf&\\xdd|\\x80\\xd9IT\\n\\xfcp\\xf8\\xcb\\xca\\n\\xe6\\xca2\\xf5\\xcc"k\\xac\\x84e\\x06\\xbb\\xe6\\x1c\\xc6\\x80\\x05&V\\xb2\\xba\\xa2]\\x13bG\\xea\\xe4L\\xa1\\xc0\\xd7\\xa70\\xf5E\\xb8\\xddrP\\xbfc\\x86,\\x0e~7y\\xb6\\xea\\x83\\x7f\\\\F\\xaa%\\xdd\\x82\\xeb\\xd0\\xa9E\\xd3\\xd5N\\xf8 \\xd7\\x0646}\\xb1\\xd9BB\\x1aC\\x1f\\xbc\\xe1e\\x8b\\xb9\\x83\\x8f\\x96_\\x1c\\x10x\\xf1\\xce\\xd9\\xe2~Mfo\\xb2\\xb6\\xd1A\\x85\\x02\\xc2\\xf1\\x1c_J\\xcb\\xc9\\x1e\\xfd\\x05\\xa0\\xc0\\x0e\\xa4\\xf6\\xc6BI,\\x0c\\xb1\\xdc\\xca4?\\xae\\x87\\x82e\\xde\\x04\\x04\\x0c\\xbb\\xb8\\xd7x\\xda\\x05\\xfd\'', '6204d73e7bdc82005de77d6a0bef06a4a1e0952085547fed618babd89c2151c299c58a55030fe1446cd7e48677a6fb6008cd8633e0d41da947b97829b

Log successfully inserted in database.
Encryption time in ms:  3.738880157470703
Hashing time in ms:  0.010967254638671875
['37', '41.644528000', '192.168.81.151', '66.218.84.140', 'f4:4d:30:49:1e:01', 'BestItWo_01:d1:64', '41328', '443', "b'\\x00\\xcf\\xe11\\x97~\\xd8dhm\\x8aqf\\xe5\\xb8G\\x05\\x92\\x192\\x9fA\\x91:\\xfa\\xc4=7g\\xa1\\xc4\\xb7l\\xda\\xb1u\\xea9\\xdd\\xc8\\x7f\\xa7\\xac\\xf4\\x1d\\x8e\\x05\\xfc\\x19K-1\\xe92\\xddY\\xfc\\x14\\x9b\\x0f\\x95^\\x8a\\x0c\\x8eq3\\xce\\xef\\x02\\x85\\xe9\\xff^\\xb1\\xf1S\\xc7G\\xf7x0\\xa8KA:\\xdd$Q\\xed*\\x07tt\\xcd\\x9b\\x82D\\xa6\\x80\\x15\\x11\\xd4\\xfc\\xaa\\xf7\\xa0eT\\x81\\x02\\x08\\xb1\\xe4\\x0eR\\n\\x1bq\\xa3\\x9b\\x03\\xb4\\xf7\\xfb\\xd7\\xcf\\xa44\\x87SU\\x1df\\xa4G\\x1e\\x93\\xf8\\x89\\xfd\\x08o\\x80\\xb8,>,\\xcf\\x06\\xc1\\xf1\\x9bU\\xe3<\\xdd)T\\xcaI,zru\\x0cS1L?I'", '5f0704a18be4d86f3faa4dc09915b6254a0a90cb3c4af7e178a69e1f76a640f495c6b0079fa2402d7c1b20797e130c0de925667ff6491682dd528663c8c4caab']
Log successfully inserted in data

Encryption time in ms:  5.21397590637207
Hashing time in ms:  0.012874603271484375
['48', '43.708188000', '10.0.47.1', '10.0.47.2', 'f4:4d:30:49:1e:01', 'fa:16:3e:94:7c:0d', '53', '30332', 'b\'\\x01`\\xb0\\x15\\xae\\xa0E?\\xd3(\\x08\\xb3\\xe3`b\\xf1\\x87\\xa7$\\xf4\\t\\\\\\xec\\xdc\\t\\xcc\\xc4\\x9e\\xbf\\xd6\\x1a\\x03"\\xf1+\\xb7@\\xbc\\x02\\xd9\\xbb*,\\x9br$\\x99\\xba\\x80\\xd0z\\x91=x=Bz%P\\xbb\\xc8$\\xaf\\xb8\\xef\\xb7\\xa1d\\x98\\xb0\\xddv:\\xd6.\\x94,}K(t%\\xf4f\\x0e\\xd9Ih\\xb7\\xa1x>;\\xe7\\xda\\x19\\x83\\xfc\\xb0\\xadu\\xe1\\xed\\x070F\\xf1Vs\\xcd\\xab\\xdc\\xd8\\xe0>\\xe0\\xc4\\xc0\\xb0,:\\xbc\\xbe\\x02\\xb4\\xfe\\xd9\\x9b\\x86%\\xb8\\xeb\\x0e(\\\'\\xda\\x0b\\x80\\xb6\\xef\\x89FT\\x92\\x192\\x91<\\xd9\\xae\\xb7\'', 'af3968be9a474b8c377a496107693b4126343f4b73a700e7c9f418e1aad676e7bdf41d426b4d78b9e45701d71447ae8ad43dc5f2f60b1d4147e68efe641411a8']
Log successfully inserted in database.
Encryption time in ms:  6.011009216308594
Hashing time in ms:  0.008821487426757812
['49', '43

Hashing time in ms:  0.011920928955078125
['58', '46.655433000', 'fa:16:3e:94:7c:0d', 'f4:4d:30:49:1e:01', '10.0.47.1', '10.0.47.2', '443', '36235', 'b"\\x00\\x81\\xbe\\xd82\\x9e\\xdd\\x0708\\xc6\\xad\\xf1\\xb4\'o\\xc6\\x1a(\\x9b\\xfa\\xb1\\xc6@\\xd2\\xafe\\xc3=\\xfdfF;Pb\\rG\\xf0w\\xad\\xfe\\xf8\\x84\\xe1eS\\x86\\xd7h\\xca\\xc5\\xe6\\x8a\\xb6m\\xfe\\x96}\\xe9\\x1e\\n\\xe9\\xea\\x91\\x1f~\\x05\\x9a>\\x88\\xe2\\x808\\x07\\x89\\t\\x8e>\\xbaw\\x06^\\x83x\\x10\\x13Z\\x04\\xc0\\x1d\\x9c\\x04\\xd0a\\xa7\\xf7\\xdab\\x02\\xdc\\xc3>\\x93\\xbe\\xc7\\x1b\\xdb\\x8a\\xe9\\x8b\\xf7A\\xc9\\xf3D\\x86+0N o_\\xdf\\xe7M\\\\\\xaa&\\xdd>\\xc7\\x18\\xdd\\xa3Y\\x83\\x9f\\x13\\xf1\\x03\\x864m\\x8d\\xb6\\xce\\x98\\xdbw\\xefT\\xa2\\xcd"', '11d8ba3c83b2740a5e2bfbd4aeac9ce12442e9d6650515f669350abcc06ef61bf808a0e361926186fe303e75997192c5f7f1beaecd6870ff1572c4815200d389']
Log successfully inserted in database.
Encryption time in ms:  5.278110504150391
Hashing time in ms:  0.015735626220703125
['59', '46.762256000',

Hashing time in ms:  0.016689300537109375
['76', '47.075652000', '192.168.81.151', '104.25.218.21', 'f4:4d:30:49:1e:01', 'BestItWo_01:d1:64', '36235', '443', 'b\'\\x00\\x08dI\\xb0\\x1e\\x86[\\xb5\\xc8\\xf6\\x0bG\\x1e\\xe14\\xed\\xf0;\\xbd\\xdet\\x04T2x\\xc2_\\x08\\xba\\xab\\x00\\x8d\\x88\\x1f\\xef\\xa7=T\\x8b\\xdc\\\'6\\x82\\xa9m\\xca\\xa6;n\\xa9\\xb5\\x84\\xd2\\xd5T+\\xfe\\xd2F\\x94\\x84;z\\xd9\\x06\\xd3uc\\xe2\\x11\\xa4\\xf7\\x1e\\xb9Ac\\xa6V\\x84^\\\\\\x16\\xe3J\\xe8"U\\x82\\xc2\\x80\\xd9G=\\xc2A\\xda\\xce\\xd4\\xca\\xc2_\\xc5\\xbc`h\\x97\\x96\\xffF\\xe3iqJd\\xf1\\xee\\x97WQ\\x978\\xa2\\xcf\\xc4\\xd2\\xe2\\xe6\\xc0C\\x18\\xd7P(\\x9a^\\xce\\x1c\\x1cVI\\x8eY\\x93W\\xfe\\xe5w\\x05\\xf8\\xfe\\xd9\\xa9\\xe0`\\xf0d\\xa8\\xe0\\x1e4\\xd3\\x94\\x8b<\\xfa\\xcf\\xeeg\\xf3\\x1f\'', '6a9d5a62046370997e01c4dd70ca67d007059ec36dff39cb0e76fd285be97ca6b478ce634f3aceaa2cab91b58e5a6189c1a82e7adbeaee52d50b2ccde72b03bc']
Log successfully inserted in database.
Encryption time in ms:  5.45811653137207
Hash

Hashing time in ms:  0.022172927856445312
['87', '53.951456000', '192.168.81.151', '66.218.84.140', 'f4:4d:30:49:1e:01', 'BestItWo_01:d1:64', '41328', '443', 'b\'\\x00\\x87E\\x89 N\\x88\\x88\\xb1\\xd2\\x95\\x8f\\xc9ge3\\x1f\\xf8\\xf8\\x89"\\x0b]\\xe3\\xc2\\xc0\\xa9\\xe2\\x87\\x96\\x81\\xa1\\x92\\xd2V\\xb0\\x07\\x98O\\xefi\\x87\\xb6l\\x9d\\xb8\\xb2m\\xbf\\x83]I2\\x9f6\\xb0\\x832 \\x16\\xd3K\\xaf\\xfd\\xbc\\xdeO\\xf6\\x7f`v\\xcbL\\xe7\\x8f\\xf4NJ`\\x85\\xc5\\xf6\\x15S\\x08\\x92\\xa8\\xa35\\xafTSj+(\\x8bx\\xc5\\xf0n\\xefke\\xce\\xb3\\x8fzl\\xb6n23\\x1b\\xbf\\xcbgM\\xe3\\x95\\xf6\\x15\\xd1\\xfd\\xe7qkK\\x1f\\x19N\\xa4\\x0b0M\\xe8\\xe5\\x95M\\x88ri\\xa3\\x07\\x95\\x8b\\xb0\\xdbqKdn\\xf2,b\\x19WZ]h \\x98\\xdf\\x01\\xc0\\x1d\\xab\\xaf\\xb1\\xa9\\xf1<\'', '0ef0b81656ecd3a93e9f9809f13465106266cd4b190caf3918838433d295195bcebe0702c0043c62eecc5519b33d07d0ff9a6ea5c1d82cfa0c2ca739a2bb3565']
Log successfully inserted in database.
Encryption time in ms:  11.490106582641602
Hashing time in ms:  0.01692

Encryption time in ms:  4.058122634887695
Hashing time in ms:  0.0209808349609375
['101', '85.171708000', '66.218.84.140', '192.168.81.151', 'BestItWo_01:d1:64', 'f4:4d:30:49:1e:01', '443', '41328', 'b"\\x00Vh\\xc378\\xf3\\xd4\\xed\\\\}Bq\\n]\\xa9\\x80D\\xee\\x88\\xe4=M\\xcfRd\'_\\xf1\\x1a?\\xdf6`\\xd9\\xe11\\xc1\\xde]uJ\\x9a\\xe1{\\x85\\xbdX\\xf9\\x8aG\\x03\\xaf\\xf8\\xdbs\\xdc\\x0e\\xa8m^\\x94\\x18\\xf1xw\\xf3\\x8c\\x16\\x9a\\xaa\\x0cZ\\xc2\\xd8\\x8c\\x87X\\xd7\\x11\\xb7\\xe0(l 4\\xf9\\xa1y\\x9f\\t2{^Pwl\\xbaz\\x1d+\\xfa|\\x99\\xb2\\x16\\xf4%Q\\xd2\\xf77.\\x12q`<\\xacNA:{d\\xed\\x90\\xc7\\xf6\\x00\\x06\\xd7jl\\xc3\\xae\\x99\\x9buD:\\x196\\xedx\\x13\\xf2\\xbd\\xb4\\xf8g\\xa6C|\\xfc\\xf2}\\xff\\n\\xac\\xc5c\\xce\\xdef\\xa8\\x86~\\x9b\\xee(\\xa43"', '120c871303db9e391233eb87cbd1a0016ad95bf167b33b106efd37969813e5e641347f1f195a1339c26d413befb48f210e9c78b5830ad8d5403961727a086589']
Log successfully inserted in database.
Encryption time in ms:  6.041049957275391
Hashing time in ms:  0.01788

Encryption time in ms:  6.002902984619141
Hashing time in ms:  0.010013580322265625
['112', '106.538691000', '192.168.81.151', '104.25.218.21', 'f4:4d:30:49:1e:01', 'BestItWo_01:d1:64', '36235', '443', "b'\\x01qEf\\xfe\\xb6Bj\\xd5\\xc70\\xbb\\x91f1\\x83\\x9b7\\x99\\xe6V\\xe2`\\x006:^9]&\\xc1\\xfd\\xb6\\x95\\xc2\\xffO\\xf4\\xad\\xf3\\x1dc\\xcdN\\x1e\\x0b\\x18.f\\xf2;f\\x8a\\xb7\\x06\\xa0G\\xe8\\xfdD\\xb3\\x89/\\xbd?2\\xc9<\\xb5RoZer2v<s\\x8a\\x03\\xac\\xed\\x04N\\xa0h\\xbe\\x1d\\xf4\\xe2R\\xddg\\x19\\xb8H\\x1a\\x9f\\x1a\\xba\\xd2\\x98\\x85\\xb8\\x93\\xf4\\xd6Q\\xf2=\\x92\\xc16\\x9f\\xa8%\\x9f\\x0f\\xbf\\r\\xeb\\xbc \\xca\\xa4\\x91\\xf6\\x02\\xb1Dy\\xe1\\xfd\\x81\\x8e\\xa1\\xffI2OB\\x9e\\x97}$\\x19\\xe0f\\xa7+\\xc3\\xa4\\xab\\xcc&\\x1bB\\x98\\x08Peu\\xeeb\\x0b\\xaa\\x0b\\xd6\\xf2\\xde\\xf2'", '3191f979b687c76c2b181d2a1e3614a2fcc3f1b4a2bf33ce4747bc0be484c9b0778ccffbed1b137b73e85da2d6d3bc047becbeb529b58dd6a351af41223dbb7d']
Log successfully inserted in database.
Encryption time in ms:  4.6

True
Insertion time in ms:  0.9217262268066406
True
1
